In [5]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline 

## Hall of Fame : Nominations and Inductions

### Table: Hall of Fame

#### For this analysis, we will make observations on patterns of how Hall of Fame nominations and inductions have occured over the years and see if there are any interesting revealations.

In [6]:

#loading the hall of fame table
hof = pd.read_csv('./core/HallOfFame.csv')

FileNotFoundError: [Errno 2] No such file or directory: './core/HallofFame.csv'

In [ ]:
hof.head()

In [ ]:
#Count of HOFs by year

plt.figure(figsize=(15,6))
hof_countplot = sns.countplot(x='yearid',data=hof,hue='inducted')

hof_countplot.set_xticks(hof_countplot.get_xticks(), hof_countplot.get_xticklabels(), rotation = 90, ha = 'center', fontsize = 'x-small')
hof_countplot.set_title("Hall of Fame: Nominations and Inductions over years")
hof_countplot.grid(True)

### Observation : 1958 was the year with maximum hall of fame values but no one was inducted.

### Reason: In 1958, no players received the required 75% of BBWAA ballots for election to the Hall. The top vote getter was future electee Max Carey, who was named by 51.1% of the writers. Because the Veterans Committee held no vote, there was no induction ceremony in Cooperstown.

### Observation : 2006 was the year with maximum inductions.

### More Info: In 2006, A record 18 inductees made up the largest single class in history.

This analysis could be useed to study hall of fame patterns in future and include them in performance and incentive related studies.

## Analysis 2 : 

### Table: Salaries and Teams

#### For this analysis, we will aim to find out how has mean salaries changed over the years for teams and what are the teams with maximum salaries. We will also find out whether or not one of the reasons for a team to be a team that has maximum salaries is their consistent or better performance in games in terms of wins and ranks.

In [ ]:
sal = pd.read_csv("./core/Salaries.csv")

In [ ]:
sal.head()

In [ ]:
#lets look at the mean salaries over the years
sal_by_year = sal.groupby(by='yearID').agg({'salary':'mean'}).reset_index()
new_columns = ['yearID','mean_salary']
sal_by_year.columns = new_columns

In [ ]:
sal_by_year['mean_salary'] = pd.to_numeric(sal_by_year['mean_salary'])


In [ ]:
pd.set_option('display.float_format', str)

In [ ]:
team_sal_scatterplot = sns.scatterplot(x='yearID',y='mean_salary',data=sal_by_year);
team_sal_scatterplot.set_title("Team Mean Salaries over the years")

### Looks like a normal behaviour, the mean salary has increased over the years. It is mainly because of the 'money' reasons like:
    - More funds 
    - Sport has become more of an entertainment thing and more and more 'viewers' keep adding up
    - More sponsor deals

In [ ]:
#lets find out teams with maximum average salary every year

sal.salary.max()

In [ ]:
#mean salaries for each team by year


team_mean_sal = sal.groupby(['yearID','teamID']).agg({'salary':'mean'}).reset_index()
team_mean_sal = team_mean_sal.rename(columns={'salary': 'mean_salary'})

In [ ]:
#max mean salaries by year
year_max_sal = team_mean_sal.groupby(['yearID']).agg({'mean_salary':'max'}).reset_index()

In [ ]:
#joining the two tables above to find out team id with maximum mean salary for each year

year_team_sal = pd.merge(team_mean_sal,year_max_sal,on=['yearID','mean_salary'])

In [ ]:
#lets look at the team ids and their counts as teams with maximum mean salary
max_salary_counts = year_team_sal['teamID'].value_counts()

It can be seen that NYA has been the team with maximum mean salary for maximum number of years. Therefore, further analysis concerns NYA

In [ ]:
#nya with its mean salaries over the years:

nya_mean_sal_over_years = year_team_sal[year_team_sal['teamID']=='NYA']

In [ ]:
nya_salary_lineplot = sns.lineplot(x='yearID',y='mean_salary',data=nya_mean_sal_over_years)
nya_salary_lineplot.set_title("Mean salary over the years for NYA")
nya_salary_lineplot.grid(True);

Let's find out counts of teams being the teams with maximum mean salaries over the years.

In [ ]:
#teams with maximum salary and their counts of being teams with max salary from 1985-2016
team_and_sal = pd.DataFrame(year_team_sal['teamID'].value_counts()).reset_index()

In [ ]:
explode = [0.1 if teamID == 'NYA' else 0 for teamID in list(team_and_sal.teamID.unique())]

rank_pichart = plt.pie(team_and_sal['count'],labels=team_and_sal['teamID'],explode=explode,autopct='%1.0f%%')
plt.title("Teams being teams with Max Average Salaries");


In [ ]:
#let's find out NYA's performance over the years and see if them being the team with max salary is justified by their performance.

In [ ]:
teams = pd.read_csv("./core/Teams.csv")

In [ ]:
team_wins = teams.groupby(by='teamID').agg({'W':'sum'}).reset_index()

In [ ]:
#This shows that NYA is on postition second in number of wins
team_wins.sort_values('W',ascending=False)

In [ ]:
#lets see NYA's rank distribution
nya_stats = teams[teams['teamID']=='NYA']

In [ ]:
plt.figure(figsize=(12,6))
nya_rank_countplot = sns.countplot(x='Rank',data=nya_stats)
nya_rank_countplot.set_title("NYA rank distribution")
nya_rank_countplot.grid(True)

In [ ]:
# Turns out, performance could be a reason of the highest average salary. Ranked by the wins NYA is on 2nd position and when rank distribution is plotted, rank 1 has the highest
#count as compared to other counts

### Could the reason for NYA being the team with highest average salary be their performance? i.e. has their mean salary increased as a result of their better performance in terms of wins and ranks in the past?

    ### It *could* be 
        - The lineplot for median salaries for nya shows a genereal increasing trend with two dips between 2005 and 2015
        - from the years 1985-2016 NYA has been the team with maximum average salary among all teams for 59% of the time, the pie chart shows that
        - The rank distribution over the year i.e. the barplot shows that NYA has been ranked 1st 47 times and have barely left the top 5 rankings

        these are indicators of good performance and thus, one of the reasons that it is a team with maximum average salary could be their performance.

This analysis could be really resourceful for the times when grants are to be given to the teams based on the needs. For example, the teams who have performed good enough but salaries do not correspond to their performance can be analysed and one of the starter points for that analysis could be this one. Also, this analysis can be used by anyone trying to study the money distribution and imbalances in salaries.

# Analysis 3:
# Complete Games vs Career Length of Pitchers

## Introduction
For this analysis, we will make the claim that pitchers have longer careers the less complete games they throw. This would correlate because there would be less damage and stress on their arms. We will analyze data to compare and see if we're correct.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
try:
    import seaborn as sns
except ImportError as e:
    %pip install seaborn
    import seaborn as sns
sns.set(style="whitegrid") 

pitching_data = pd.read_csv('./core/Pitching.csv') 

## Data Preparation


In [ ]:
# total cgs
cg_yearly = pitching_data.groupby('yearID')['CG'].sum().reset_index()

# career length
career_length = pitching_data.groupby('playerID')['yearID'].agg(['min', 'max'])
career_length['career_length'] = career_length['max'] - career_length['min']

career_length['yearID'] = career_length['max']
career_length_average = career_length.groupby('yearID')['career_length'].mean().reset_index()

cg_career_length_analysis = pd.merge(cg_yearly, career_length_average, on='yearID')

First, we load the data and calculate the key metrics for our analysis: the number of complete games per year and the average career length of pitchers.

## Data Analysis and Trends


In [ ]:
plt.figure(figsize=(14, 7))
ax = sns.lineplot(x='yearID', y='CG', data=cg_yearly, label='Total Complete Games', color='skyblue')
plt.ylabel('Total Complete Games')
ax2 = plt.twinx()
sns.lineplot(x='yearID', y='career_length', data=career_length_average, label='Average Career Length (Years)', color='coral', ax=ax2)
plt.ylabel('Average Career Length (Years)')
plt.title('Complete Games vs. Career Longevity Over Years')
plt.legend(loc='upper left')
plt.show()

With the metrics calculated, we will observe the trends over time to determine how the role and usage of pitchers have evolved.

In [ ]:
plt.figure(figsize=(14, 7))
sns.histplot(career_length['career_length'], bins=30, kde=True, color='purple')
plt.title("Distribution of Pitchers' Career Lengths")
plt.xlabel('Career Length (Years)')
plt.ylabel('Number of Pitchers')
plt.show()

## Analysis and Conclusion


The data shows a significant decrease in the number of complete games over the years, which aligns with the broader trend in baseball towards more specialized roles, such as relief and closing pitchers. This strategy may be a response to protect pitchers from injury and fatigue, which can result from the high physical demand of pitching complete games.

However, the data does not indicate a clear increase in career longevity despite the reduction in complete games. This could suggest that while pitchers are being protected more during games, other factors may be influencing career length. These different factors could include things like better sports medicine, better training programs, a more talented and larger player pool, or simply the natural physical limitations of pitching over many seasons.

Moreover, advances in sports medicine and surgery may mean that injuries that once ended careers can now be treated effectively, allowing pitchers to return to play. However, the stress of pitching, particularly at high velocities, still takes a toll on the body, and this may limit the potential for significantly longer careers.

It's important to recognize that the decline in complete games is a strategic choice that reflects more of a change in strategy in the game of baseball rather than a decline in pitcher's ability or longevity. 

# Analysis 4: 
# Pitching Metrics Analysis: ERA vs Wins and WHIP vs Wins

## Introduction
In this analysis, we will first see if a lower WHIP means a lower ERA, and visa versa. We will then compare those stats with wins to see if there is a correlation between these pitching performance metrics and wins.



In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")


pitching_data = pd.read_csv('./core/Pitching.csv')  # Replace with your actual file path

# Calculating WHIP 
pitching_data['IP'] = pitching_data['IPouts'] / 3  # Convert outs to innings pitched
pitching_data['WHIP'] = (pitching_data['BB'] + pitching_data['H']) / pitching_data['IP']


In [ ]:

# WHIP vs ERA
plt.figure(figsize=(10, 6))
sns.scatterplot(x='WHIP', y='ERA', data=pitching_data, alpha=0.6)
plt.title('WHIP vs ERA')
plt.xlabel('WHIP (Walks plus Hits per Inning Pitched)')
plt.ylabel('ERA (Earned Run Average)')
plt.show()


In [ ]:

# ERA vs Wins
plt.figure(figsize=(10, 6))
sns.scatterplot(x='W', y='ERA', data=pitching_data, alpha=0.6)
plt.title('ERA vs Wins')
plt.xlabel('Games Won')
plt.ylabel('ERA (Earned Run Average)')
plt.gca().invert_yaxis()  # Inverting y-axis as lower ERA is better
plt.show()


In [ ]:

# WHIP vs Wins
plt.figure(figsize=(10, 6))
sns.scatterplot(x='W', y='WHIP', data=pitching_data, alpha=0.6)
plt.title('WHIP vs Wins')
plt.xlabel('Games Won')
plt.ylabel('WHIP (Walks plus Hits per Inning Pitched)')
plt.gca().invert_yaxis()  # Inverting y-axis as lower WHIP is better
plt.show()



## Analysis and Conclusion
The visualizations suggest that both ERA and WHIP are indicators of a pitcher's success in terms of wins. However, the relationship is not perfectly linear, indicating that other factors also play a significant role in a pitcher's success. 

It does appear that for the most part, lower ERA correlates with lower WHIP, with some general outliers. These could be due to various reasons, such as defense, high walk pitures that get out of jams, or many other things.

From the scatter plots, we can observe that a lower ERA tends to correspond with a higher number of wins, which aligns with conventional baseball wisdom. Similarly, a lower WHIP also tends to correlate with more wins, supporting the claim that pitchers who allow fewer batters to reach base are more likely to be successful.

In conclusion, while both ERA and WHIP are valuable metrics, neither can be the sole predictor of a pitcher's success. There ends up being many other factors in baseball that affect a pitcher's success and the team's ability to win. These things include run support, defense, innings pitched, and all sorts of other factors. The complexity of baseball and the numerous variables affecting game outcomes mean that these metrics should be considered alongside other data to evaluate a pitcher's performance thoroughly.
